In [1]:
# !pip install ensemble_boxes

In [1]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

from os.path import getsize
print(getsize('/opt/ml/code/Swin-Transformer-Object-Detection/work_dirs/swin.csv'))
print(getsize('/opt/ml/code/Swin-Transformer-Object-Detection/work_dirs/swin_TTA.csv'))

823218
798474


In [3]:
# ensemble csv files
submission_files = ['/opt/ml/code/Swin-Transformer-Object-Detection/work_dirs/swin.csv',
                    '/opt/ml/code/Swin-Transformer-Object-Detection/work_dirs/swin_TTA.csv']
submission_df = [pd.read_csv(file) for file in submission_files]

In [4]:
submission_df[0].head()

,PredictionString,image_id
0,6 0.08392282 184.24693 365.6062 210.95854 398....,batch_01_vt/0021.jpg
1,1 0.7738614 4.44707 246.07396 307.3681 456.791...,batch_01_vt/0028.jpg
2,1 0.9999099 69.92787 166.90848 379.1483 379.9666,batch_01_vt/0031.jpg
3,1 0.09910974 296.37112 159.159 363.94086 226.8...,batch_01_vt/0032.jpg
4,0 0.27640808 37.842575 459.38425 84.2498 474.1...,batch_01_vt/0070.jpg


In [5]:
image_ids = submission_df[0]['image_id'].tolist()

In [6]:
annotation = '/opt/ml/input/data/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [7]:
prediction_strings = []
file_names = []
iou_thr = 0.6
skip_box_thr = 0.0001

weights = [4, 1] # 모델 가중치

for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        if len(predict_list)==0 or len(predict_list)==1:
            continue
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
    if len(boxes_list):
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

In [8]:
# submission = pd.DataFrame()
# submission['PredictionString'] = prediction_strings
# submission['image_id'] = file_names
# submission.to_csv('/opt/ml/output/submission_ensemble.csv')

# submission.head()

In [9]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('/opt/ml/code/Swin-Transformer-Object-Detection/work_dirs/submission_ensemble.csv')

submission.head()

,PredictionString,image_id
0,8 0.9999874830245972 173.00685119628906 233.86...,batch_01_vt/0021.jpg
1,8 0.9992231130599976 160.72933959960938 62.590...,batch_01_vt/0028.jpg
2,1 0.999903678894043 69.87248229980469 166.9575...,batch_01_vt/0031.jpg
3,8 0.9978682 25.327510833740234 143.09451293945...,batch_01_vt/0032.jpg
4,8 0.9999496340751648 150.7216033935547 2.01026...,batch_01_vt/0070.jpg
